# Cleaning of Sales Monitoring Data

Notebook by Dominic Dayta, for Aroodai and Lean Retail Labs.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
clf = linear_model.PoissonRegressor()

Load data

In [ ]:
sales = pd.read_excel (r'/content/drive/MyDrive/Colab Notebooks/Data Files/SALES/DOM 2017 Sales Oct rows72550 Copy of Havaianas Sales as of Oct 2017 with notes.xlsx',
                           sheet_name = 'Sheet1', na_values=['#DIV/0!'])

sales.head()

,REGION,STORE,MONTH,RECEIPT DATE,BARCODE,STYLE,COLOR,COLOR CODE,SIZE,SEASON,...,SRP,DISC %,DISC $,SALES AFT DISC,NET SALES W/O GST,BRAND,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,SINGAPORE,BG,1.0,2017-01-01,7.890558e+12,SLIM,SAND GREY/LIGHT GOLDEN,2719.0,356.0,2010.0,...,39.9,0.0,0.0,39.9,37.29,HAVAIANAS,NaN,NaN,NaN,NaN
1,SINGAPORE,BG,1.0,2017-01-01,7.890558e+12,SLIM,SAND GREY/LIGHT GOLDEN,2719.0,390.0,2010.0,...,119.7,0.0,0.0,119.7,111.87,HAVAIANAS,NaN,NaN,NaN,NaN
2,SINGAPORE,BG,1.0,2017-01-01,7.890558e+12,TOP,MARINE BLUE,2711.0,356.0,2010.0,...,32.9,0.0,0.0,32.9,30.75,HAVAIANAS,NaN,NaN,NaN,NaN
3,SINGAPORE,BG,1.0,2017-01-01,7.890732e+12,SLIM,WHITE,1.0,378.0,2009.0,...,39.9,0.0,0.0,39.9,37.29,HAVAIANAS,NaN,NaN,NaN,NaN
4,SINGAPORE,BG,1.0,2017-01-01,7.890732e+12,SLIM,NAVY BLUE,555.0,356.0,2010.0,...,39.9,0.0,0.0,39.9,37.29,HAVAIANAS,NaN,NaN,NaN,NaN


Rename columns and drop blank spaces in the Excel file.

In [ ]:
sales.columns

Index(['REGION', 'STORE', 'MONTH', 'RECEIPT DATE', 'BARCODE', 'STYLE', 'COLOR',
       'COLOR CODE', 'SIZE', 'SEASON', 'SOLD QTY', 'SRP', 'DISC %', 'DISC $',
       'SALES AFT DISC', 'NET SALES W/O GST', 'BRAND', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'],
      dtype='object')

In [ ]:
sales.columns = ['region', 'store', 'month', 'receipt_date', 'barcode', 'style',
                     'color', 'color_code', 'size', 'season', 'quantity_sold','srp',
                     'discount_pct', 'discount_price','sales_after_discount', 'net_sales_before_taxes',
                     'brand', 'unnamed1', 'unnamed2', 'unnamed3', 'unnamed4']

sales.head()

,region,store,month,receipt_date,barcode,style,color,color_code,size,season,...,srp,discount_pct,discount_price,sales_after_discount,net_sales_before_taxes,brand,unnamed1,unnamed2,unnamed3,unnamed4
0,SINGAPORE,BG,1.0,2017-01-01,7.890558e+12,SLIM,SAND GREY/LIGHT GOLDEN,2719.0,356.0,2010.0,...,39.9,0.0,0.0,39.9,37.29,HAVAIANAS,NaN,NaN,NaN,NaN
1,SINGAPORE,BG,1.0,2017-01-01,7.890558e+12,SLIM,SAND GREY/LIGHT GOLDEN,2719.0,390.0,2010.0,...,119.7,0.0,0.0,119.7,111.87,HAVAIANAS,NaN,NaN,NaN,NaN
2,SINGAPORE,BG,1.0,2017-01-01,7.890558e+12,TOP,MARINE BLUE,2711.0,356.0,2010.0,...,32.9,0.0,0.0,32.9,30.75,HAVAIANAS,NaN,NaN,NaN,NaN
3,SINGAPORE,BG,1.0,2017-01-01,7.890732e+12,SLIM,WHITE,1.0,378.0,2009.0,...,39.9,0.0,0.0,39.9,37.29,HAVAIANAS,NaN,NaN,NaN,NaN
4,SINGAPORE,BG,1.0,2017-01-01,7.890732e+12,SLIM,NAVY BLUE,555.0,356.0,2010.0,...,39.9,0.0,0.0,39.9,37.29,HAVAIANAS,NaN,NaN,NaN,NaN


In [ ]:
# proper date formatting

sales['receipt_date'] = pd.to_datetime(sales['receipt_date'])
sales = sales.drop(['unnamed1','unnamed2','unnamed3','unnamed4'], axis = 1)

sales.head()

,region,store,month,receipt_date,barcode,style,color,color_code,size,season,quantity_sold,srp,discount_pct,discount_price,sales_after_discount,net_sales_before_taxes,brand
0,SINGAPORE,BG,1.0,2017-01-01,7.890558e+12,SLIM,SAND GREY/LIGHT GOLDEN,2719.0,356.0,2010.0,1.0,39.9,0.0,0.0,39.9,37.29,HAVAIANAS
1,SINGAPORE,BG,1.0,2017-01-01,7.890558e+12,SLIM,SAND GREY/LIGHT GOLDEN,2719.0,390.0,2010.0,3.0,119.7,0.0,0.0,119.7,111.87,HAVAIANAS
2,SINGAPORE,BG,1.0,2017-01-01,7.890558e+12,TOP,MARINE BLUE,2711.0,356.0,2010.0,1.0,32.9,0.0,0.0,32.9,30.75,HAVAIANAS
3,SINGAPORE,BG,1.0,2017-01-01,7.890732e+12,SLIM,WHITE,1.0,378.0,2009.0,1.0,39.9,0.0,0.0,39.9,37.29,HAVAIANAS
4,SINGAPORE,BG,1.0,2017-01-01,7.890732e+12,SLIM,NAVY BLUE,555.0,356.0,2010.0,1.0,39.9,0.0,0.0,39.9,37.29,HAVAIANAS


In [ ]:
sales.to_csv(r'/content/drive/MyDrive/Colab Notebooks/Data Files/Cleaned/sales.csv', index=False)